# 1. ollama 서버 상태 확인
#### 서버 실행되지 않았을 때, ConnectionError 방지

In [1]:
import requests

# 서버 연결 체크 함수 
def check_ollama_status(base_url="http://localhost:11434"):
    try:
        response = requests.get(base_url)
        return response.status_code == 200
    except requests.exceptions.ConnectionError:
        return False

# 서버가 켜져 있는지 확인
if not check_ollama_status():
    print("[ERROR] Ollama 서버가 실행 중이지 않습니다.")
    print("터미널에서 'ollama serve'를 실행하거나, 설치 여부를 확인하세요.")


# 2. Few-shot 정형 데이터 추출기
#### 2-1. Temperature 설정 (temperature=0): 정확한 답변 위해
#### 2-2. 품질의 예시 (Examples): Few-shot의 핵심은 "모델이 패턴을 인지하게 하는 것"
#### 2-3. Prefix와 Suffix의 역할
###### - Prefix: "반드시 JSON 포맷만 출력하세요"라는 제약 사항을 걸어 모델이 "네, 알겠습니다" 같은 불필요한 서두를 떼지 않게 함
###### - Suffix: 사용자의 실제 입력값(input_mail)을 배치하고, 마지막에 추출 결과(JSON):라는 문구를 두어 모델이 바로 다음 토큰으로 {를 생성하도록 유도
#### 2-4. 일부 데이터 누락된 예시 포함: (아는 척 못 하게) Hallucination 방지

In [1]:
import json
from langchain_community.llms import Ollama
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate

# 1. 로컬 LLM 설정 (예: llama3, mistral, 혹은 한국어 특화 모델)
llm = Ollama(model="llama3", temperature=0) # TODO 추출 작업은 정밀도 중요 temperature=0 설정

# 2. Few-shot을 위한 예시 데이터 (Examples) 정의
# 모델에게 어떤 비정형 텍스트에서 어떤 JSON 결과가 나와야 하는지 학습 예시 [20260112] 예시 데이터의 JSON 중괄호를 {{ }}로 변경
examples = [
    {
        "mail_text": "안녕하세요, 구글 코리아의 데이터 분석가 김철수입니다. 2023년 5월 10일에 입사하여 업무 보고 드립니다.",
        "answer": """{{"name": "김철수", "company": "구글 코리아", "role": "데이터 분석가", "date": "2023-05-10", "summary": "입사 후, 업무 보고"}}"""
    },
    {
        "mail_text": "네이버 마케팅팀 팀장 박영희입니다. 오는 2024년 1월 15일 미팅 일정 확인 부탁드립니다.",
        "answer": """{{"name": "박영희", "company": "네이버", "role": "마케팅팀 팀장", "date": "2024-01-15", "summary": "미팅 일정 확인"}}"""
    },
    {
        "mail_text": "삼성전자 폰팔이 이재용입니다. 프로젝트는 2023년 12월 25일에 종료될 예정입니다.",
        "answer": """{{"name": "이재용", "company": "삼성전자", "role": "폰팔이", "date": "2023-12-25", "summary": "프로젝트 종료 예정"}}"""
    },
    {
        "mail_text": "현대 중고차 딜러 정몽구입니다. 차보러 언제 오시나요?",  # # TODO [중요] 누락된 데이터(예: 날짜가 없는 경우)에 "정보 없음" 혹은 **null** 출력하는 예시 포함해 Hallucination 방지
        "answer": """{{"name": "정몽구", "company": "현대 중고차", "role": "중고차 딜러", "date": "정보 없음", "summary": "방문 일정"}}"""
    },
]

# 3. 예시를 보여줄 포맷(Template) 정의
example_formatter_template = """
메일 내용: {mail_text}
추출 결과(JSON): {answer}
"""
example_prompt = PromptTemplate(
    input_variables=["mail_text", "answer"],
    template=example_formatter_template,
)

# 4. Few-shot 프롬프트 구성
few_shot_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    # 지시사항 부분에 예시 형식을 적을 때도 {{ }} 사용
    prefix="사내 메일에서 정보를 추출하여 {{\"name\": \"...\", \"company\": \"...\"}} 형식의 JSON으로 답하세요.",
    # prefix="다음 사내 메일 텍스트에서 이름, 회사, 역할, 날짜를 추출하여 JSON 형식으로 답변하세요. 반드시 JSON 포맷만 출력하세요.",
    suffix="메일 내용: {input}\n추출 결과(JSON):",
    input_variables=["input"],
    example_separator="\n\n",
)

# 5. 실행 예시
input_mail = """
안녕하세요.

경영지원팀 이세돌 입니다.

엑셀 파일도 작성 후 회신 부탁 드립니다.

감사합니다.
"""
final_prompt = few_shot_prompt.format(input=input_mail)

print("--- [생성된 프롬프트] ---")
print(final_prompt)

print("\n--- [LLM 추출 결과] ---")
response = llm.invoke(final_prompt)
print(response)

# 6. JSON 파싱 테스트 (실제 서비스 활용 시)
try:
    data = json.loads(response)
    print("\n[파싱 성공]")
    print(f"이름: {data['name']}, 주요내용: {data['summary']}, 날짜: {data['date']}, 직책: {data['role']}, 회사: {data['company']}")
except Exception as e:
    print("\n[파싱 실패]", e)

/opt/anaconda3/envs/data_analysis/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/var/folders/b2/vx12c8bs3bgfsxvbxwr9kddm0000gn/T/ipykernel_29390/3918353348.py:7: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the `langchain-ollama package and should be used instead. To use it run `pip install -U `langchain-ollama` and import as `from `langchain_ollama import OllamaLLM``.
  llm = Ollama(model="llama3", temperature=0) # TODO 추출 작업은 정밀도 중요 temperature=0 설정


--- [생성된 프롬프트] ---
사내 메일에서 정보를 추출하여 {"name": "...", "company": "..."} 형식의 JSON으로 답하세요.


메일 내용: 안녕하세요, 구글 코리아의 데이터 분석가 김철수입니다. 2023년 5월 10일에 입사하여 업무 보고 드립니다.
추출 결과(JSON): {"name": "김철수", "company": "구글 코리아", "role": "데이터 분석가", "date": "2023-05-10", "summary": "입사 후, 업무 보고"}



메일 내용: 네이버 마케팅팀 팀장 박영희입니다. 오는 2024년 1월 15일 미팅 일정 확인 부탁드립니다.
추출 결과(JSON): {"name": "박영희", "company": "네이버", "role": "마케팅팀 팀장", "date": "2024-01-15", "summary": "미팅 일정 확인"}



메일 내용: 삼성전자 폰팔이 이재용입니다. 프로젝트는 2023년 12월 25일에 종료될 예정입니다.
추출 결과(JSON): {"name": "이재용", "company": "삼성전자", "role": "폰팔이", "date": "2023-12-25", "summary": "프로젝트 종료 예정"}



메일 내용: 현대 중고차 딜러 정몽구입니다. 차보러 언제 오시나요?
추출 결과(JSON): {"name": "정몽구", "company": "현대 중고차", "role": "중고차 딜러", "date": "정보 없음", "summary": "방문 일정"}


메일 내용: 
안녕하세요.

경영지원팀 이세형 입니다.

엑셀 파일도 작성 후 회신 부탁 드립니다.

감사합니다.

추출 결과(JSON):

--- [LLM 추출 결과] ---


ConnectionError: HTTPConnectionPool(host='localhost', port=11434): Max retries exceeded with url: /api/generate (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x1479469b0>: Failed to establish a new connection: [Errno 61] Connection refused'))